<a href="https://colab.research.google.com/github/siddharthpelkhn/machine/blob/main/car-parking-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Generation**

In [3]:
import numpy as np

# Generating features for difficult parking scenario
np.random.seed(42)
X_difficult = np.random.rand(500, 3)
X_difficult[:, 0] = np.random.uniform(0, 0.2, 500)  # Short distance to curb
X_difficult[:, 1] = np.random.uniform(0, 0.2, 500)  # Short distance to cars
X_difficult[:, 2] = np.random.uniform(60, 90, 500)  # Awkward angle of approach

# Generating features for easy parking scenario
X_easy = np.random.rand(500, 3)
X_easy[:, 0] = np.random.uniform(0.5, 1, 500)  # Ample distance to curb
X_easy[:, 1] = np.random.uniform(0.5, 1, 500)  # Ample distance to cars
X_easy[:, 2] = np.random.uniform(0, 45, 500)  # Comfortable angle of approach

## **Concatenation and Label Generation:**

In [4]:
# Concatenating the datasets
X = np.vstack((X_difficult, X_easy))

# Generating labels (0 for difficult scenario, 1 for easy scenario)
y = np.concatenate((np.zeros(500), np.ones(500)))

##**Data Splitting and Normalization:**

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize/Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# **Model Definition, Training, and Evaluation:**

In [6]:
import tensorflow as tf

# Define a simple neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

Epoch 1/10
23/23 [==============================] - 1s 13ms/step - loss: 0.6282 - accuracy: 0.7861 - val_loss: 0.5235 - val_accuracy: 1.0000
Epoch 2/10
23/23 [==============================] - 0s 3ms/step - loss: 0.4589 - accuracy: 1.0000 - val_loss: 0.3709 - val_accuracy: 1.0000
Epoch 3/10
23/23 [==============================] - 0s 3ms/step - loss: 0.3147 - accuracy: 1.0000 - val_loss: 0.2362 - val_accuracy: 1.0000
Epoch 4/10
23/23 [==============================] - 0s 4ms/step - loss: 0.1911 - accuracy: 1.0000 - val_loss: 0.1333 - val_accuracy: 1.0000
Epoch 5/10
23/23 [==============================] - 0s 3ms/step - loss: 0.1048 - accuracy: 1.0000 - val_loss: 0.0728 - val_accuracy: 1.0000
Epoch 6/10
23/23 [==============================] - 0s 3ms/step - loss: 0.0589 - accuracy: 1.0000 - val_loss: 0.0427 - val_accuracy: 1.0000
Epoch 7/10
23/23 [==============================] - 0s 3ms/step - loss: 0.0360 - accuracy: 1.0000 - val_loss: 0.0275 - val_accuracy: 1.0000
Epoch 8/10
23/23 [=

# **Convert to TensorFlow Lite:**

In [8]:
import tensorflow as tf

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to a file
with open('parking_model.tflite', 'wb') as f:f.write(tflite_model)

# **Prediction Functions:**

In [9]:
# Load the TensorFlow Lite model for parking prediction
interpreter = tf.lite.Interpreter(model_path="parking_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to preprocess user inputs
def preprocess_input(distance_to_curb, distance_to_cars, angle_of_approach):
    user_input = np.array([[distance_to_curb, distance_to_cars, angle_of_approach]])
    user_input_scaled = scaler.transform(user_input)
    return user_input_scaled.astype(np.float32)

# Function to make predictions based on user inputs
def predict_car_parking():
    # Taking user inputs for features
    distance_to_curb = float(input("Enter distance to curb (in meters): "))
    distance_to_cars = float(input("Enter distance to nearby cars (in meters): "))
    angle_of_approach = float(input("Enter angle of approach (in degrees): "))

    # Preprocess user inputs
    user_input = preprocess_input(distance_to_curb, distance_to_cars, angle_of_approach)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], user_input)

    # Run inference
    interpreter.invoke()

    # Get the model's prediction
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Display the prediction result
    if output_data[0][0] >= 0.5:
        print("Prediction: You can park the car successfully.")
    else:
        print("Prediction: It might be difficult to park the car in this situation.")

# Call the function to get user inputs and make predictions
predict_car_parking()

Enter distance to curb (in meters): 8
Enter distance to nearby cars (in meters): 1
Enter angle of approach (in degrees): 23
Prediction: You can park the car successfully.
